In [302]:
 from pytrends.request import TrendReq
from datetime import datetime as dt
import datetime

pytrends_daily = TrendReq(hl='en-US', tz=360)
pytrends_weekly = TrendReq(hl='en-US', tz=360)

In [303]:
#Parameters
#Week starts on Sunday
#Start and end must be within 90 days for daily data

start = '2022-01-02'
end = '2022-04-03'
kw_list = ['bitcoin','ethereum']


end_dt = dt.strptime(end,"%Y-%m-%d")
end_dt_last_yr = dt(end_dt.year - 1, end_dt.month,end_dt.day)
end_last_yr = end_dt_last_yr.strftime("%Y-%m-%d")

tf_daily = start +' '+ end
tf_weekly = end_last_yr + ' ' + end


In [304]:
pytrends_daily.build_payload(kw_list,cat=0,timeframe = tf_daily)
daily_data = pytrends_daily.interest_over_time()
daily_data = daily_data[kw_list]
#daily_data

In [305]:
pytrends_weekly.build_payload(kw_list,cat=0,timeframe = tf_weekly)
weekly_data = pytrends_weekly.interest_over_time()
weekly_data = weekly_data[kw_list]
#weekly_data

In [306]:
for kw in kw_list:
    daily_data[kw + '_wk'] = 0

In [307]:
for index_daily in daily_data.index:
    for index_weekly in weekly_data.index:
        if (index_daily == index_weekly):
            for i in range(7):
                try:
                    daily_data.loc[index_daily]['bitcoin_wk'] = weekly_data.loc[index_daily]['bitcoin']
                    daily_data.loc[index_daily]['ethereum_wk'] = weekly_data.loc[index_daily]['ethereum']
                except:
                    pass
            

In [308]:
daily_data['BTC SVI Adj Factor'] = daily_data['bitcoin_wk']/daily_data['bitcoin']
daily_data['ETH SVI Adj Factor'] = daily_data['ethereum_wk']/daily_data['ethereum']

In [343]:
SVI_adj_factor = daily_data[['BTC SVI Adj Factor','ETH SVI Adj Factor']].copy(deep = True)

In [346]:
for index in weekly_data.index[:-1]:
    for i in range (7):
        try:
            SVI_adj_factor.loc[index + datetime.timedelta(days = i),"BTC SVI Adj Factor"] = SVI_adj_factor.loc[index,"BTC SVI Adj Factor"]  
            SVI_adj_factor.loc[index + datetime.timedelta(days = i),"ETH SVI Adj Factor"] = SVI_adj_factor.loc[index,"ETH SVI Adj Factor"]  
        except:
            pass

In [349]:
daily_data["BTC SVI Adj Factor"] = SVI_adj_factor["BTC SVI Adj Factor"]
daily_data["ETH SVI Adj Factor"] = SVI_adj_factor["ETH SVI Adj Factor"]

In [351]:
daily_data["bitcoin Adj"] = daily_data["bitcoin"]*daily_data["BTC SVI Adj Factor"]
daily_data["ethereum Adj"] = daily_data["ethereum"]*daily_data["ETH SVI Adj Factor"]

In [354]:
daily_data.describe()

,bitcoin,ethereum,bitcoin_wk,ethereum_wk,BTC SVI Adj Factor,ETH SVI Adj Factor,bitcoin Adj,ethereum Adj
count,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,47.141304,9.336957,5.184783,1.173913,0.827610,0.888859,38.989166,8.288104
std,12.838457,2.249675,12.546992,2.861678,0.089336,0.095024,11.953633,2.250247
min,32.000000,5.000000,0.000000,0.000000,0.675676,0.750000,25.000000,5.000000
25%,39.000000,8.000000,0.000000,0.000000,0.800000,0.857143,32.166667,7.000000
50%,44.000000,9.000000,0.000000,0.000000,0.818182,0.875000,37.261538,7.875000
75%,51.000000,10.000000,0.000000,0.000000,0.843750,0.900000,41.290069,9.000000
max,100.000000,18.000000,50.000000,11.000000,1.026316,1.111111,102.631579,18.888889


In [358]:
daily_data["bitcoin Adj"] = daily_data["bitcoin Adj"]*(100/daily_data["bitcoin Adj"].max())

In [369]:
df_btc = daily_data[['bitcoin','bitcoin Adj']].copy(deep=True)
df_eth = daily_data[['ethereum','ethereum Adj']].copy(deep=True)

In [372]:
#df_btc.to_excel('Datasets/btc-googletrends.xlsx')
#df_eth.to_excel('Datasets/eth-googletrends.xlsx')